## Install Required Libraries

In [1]:
!pip install transformers[torch] datasets seqeval accelerate -U


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
import pandas as pd
from seqeval.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

c:\Users\dell g3\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load a subset of the Universal Dependencies dataset for inspection
dataset = load_dataset("universal_dependencies", "en_gum")


In [4]:
# Print the column names for the training set
print(dataset['train'].column_names)
# Convert the 'train' split of your dataset to a Pandas DataFrame
train_df = dataset['train'].to_pandas()
print(train_df.head())
print(len(train_df))

['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc']
                  idx                                               text  \
0  GUM_academic_art-1            Aesthetic Appreciation and Spanish Art:   
1  GUM_academic_art-2                         Insights from Eye-Tracking   
2  GUM_academic_art-3  Claire Bailey-Ross claire.bailey-ross@port.ac....   
3  GUM_academic_art-4  Andrew Beresford a.m.beresford@durham.ac.uk Du...   
4  GUM_academic_art-5  Daniel Smith daniel.smith2@durham.ac.uk Durham...   

                                              tokens  \
0    [Aesthetic, Appreciation, and, Spanish, Art, :]   
1                     [Insights, from, Eye-Tracking]   
2  [Claire, Bailey-Ross, claire.bailey-ross@port....   
3  [Andrew, Beresford, a.m.beresford@durham.ac.uk...   
4  [Daniel, Smith, daniel.smith2@durham.ac.uk, Du...   

                                              lemmas  \
0    [aesthetic, appreciation, and, Spanish, art, :]   

In [5]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 29.0kB/s]
c:\Users\dell g3\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dell g3\.cache\huggingface\hub\models--distilbert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message

In [6]:
unique_pos_tags = set()

# Aggregate unique POS tags from all dataset splits
for split in ['train', 'validation', 'test']:
    for example in dataset[split]:
        unique_pos_tags.update(example['upos'])

# Now that you have all unique POS tags, create the pos_tag2id and id2pos_tag dictionaries
pos_tag2id = {tag: idx for idx, tag in enumerate(unique_pos_tags)}
id2pos_tag = {idx: tag for tag, idx in pos_tag2id.items()}


In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding="max_length", is_split_into_words=True, max_length=128)

    labels = [[] for _ in range(len(examples["tokens"]))]  # Initialize labels list for each example
    for i, label_sequence in enumerate(examples["upos"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Maps tokens to their word
        label_ids = [-100] * len(word_ids)  # Initialize all labels to ignore index

        previous_word_idx = None
        label_index = 0
        for word_idx in word_ids:
            if word_idx is not None and word_idx != previous_word_idx and label_index < len(label_sequence):
                label_id = pos_tag2id.get(label_sequence[label_index])  # Use .get() to avoid KeyError
                if label_id is not None:  # Check if the label_id exists
                    label_ids[word_idx] = label_id
                label_index += 1
            previous_word_idx = word_idx
        labels[i] = label_ids
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True, remove_columns=dataset['train'].column_names)
print(tokenized_datasets)
tokenized_datasets['train']
tokenized_datasets_df = tokenized_datasets['train'].to_pandas()
print(tokenized_datasets_df.head())
print(len(tokenized_datasets_df))

Map:   0%|          | 0/4287 [00:00<?, ? examples/s]

Map: 100%|██████████| 890/890 [00:00<00:00, 5254.32 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4287
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 784
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 890
    })
})
                                           input_ids  \
0  [101, 138, 13051, 13358, 11130, 73784, 74755, ...   
1  [101, 61862, 10107, 10188, 28577, 118, 26403, ...   
2  [101, 24448, 29761, 118, 16690, 103500, 10112,...   
3  [101, 13999, 40575, 62198, 169, 119, 181, 119,...   
4  [101, 11792, 11673, 10215, 19428, 119, 39709, ...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...   
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [1, 1, 1, 1, 1, 1, 1, 1, 

## Reduce the Dataset Size

Given the objective to explore the impact of partial layer freezing on POS tagging performance, starting with a reduced dataset can help quickly gauge the feasibility and potential direction of the research. Once promising configurations are identified,the task can be scaled up to the full dataset to fine-tune and validate the findings.

*   **Shuffling**: The datasets are shuffled before reduction to ensure that the reduced dataset is a representative random sample of the original. This is important for maintaining the diversity of examples in smaller datasets.
    
*   **Seed for Reproducibility**: The `seed=42` parameter in the `.shuffle()` method ensures that the shuffling process is reproducible, meaning you'll get the same reduced dataset if you run the code multiple times.
    
*   **Adjusting Reduction Percentage**: You can modify `reduced_percentage` to any value between 0 and 1 (exclusive) depending on how much you want to reduce the dataset by. A value of `0.5` reduces the dataset to half its original size, while `0.1` would reduce it to 10%, and so on.

In [9]:
from datasets import DatasetDict

reduced_percentage = 0.1  # Target percentage to reduce each dataset to

# Function to reduce dataset size by a specified percentage
def reduce_dataset_size(dataset, percentage):
    # Calculate the number of samples to select based on the reduction percentage
    num_samples = int(len(dataset) * percentage)
    # Shuffle the dataset (with a fixed seed for reproducibility) and select the first num_samples
    reduced_dataset = dataset.shuffle(seed=42).select(range(num_samples))
    return reduced_dataset

# Reduce the size of the training, validation, and test datasets
reduced_train_dataset = reduce_dataset_size(tokenized_datasets['train'], reduced_percentage)
reduced_validation_dataset = reduce_dataset_size(tokenized_datasets['validation'], reduced_percentage)
reduced_test_dataset = reduce_dataset_size(tokenized_datasets['test'], reduced_percentage)

# Create a new DatasetDict with the reduced datasets
reduced_tokenized_datasets = DatasetDict({
    'train': reduced_train_dataset,
    'validation': reduced_validation_dataset,
    'test': reduced_test_dataset
})

# Now, you can proceed with training and evaluation using `reduced_tokenized_datasets`
reduced_tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 428
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 78
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 89
    })
})

## Define the Compute Metrics

In [10]:
def compute_metrics(p):
    predictions, labels = p
    # Flatten both the predictions and labels for evaluation
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2pos_tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [id2pos_tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists
    true_predictions = [p for sublist in true_predictions for p in sublist]
    true_labels = [l for sublist in true_labels for l in sublist]

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, true_predictions, average='macro', zero_division=0)
    accuracy = accuracy_score(true_labels, true_predictions)

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
    }


## Dataset Preparation and Baseline Model Training

In [12]:
from transformers import DistilBertForTokenClassification, TrainingArguments, Trainer

# Assuming pos_tag2id has been defined based on your dataset's unique POS tags
num_labels = len(pos_tag2id)

model_baseline = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-multilingual-cased',
    num_labels=num_labels
)

training_args_baseline = TrainingArguments(
    output_dir='./baseline_results',
    num_train_epochs=3,
    per_device_train_batch_size=128,  # Updated batch size
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer_baseline = Trainer(
    model=model_baseline,
    args=training_args_baseline,
    train_dataset=reduced_tokenized_datasets['train'],
    eval_dataset=reduced_tokenized_datasets['validation'],
    compute_metrics=compute_metrics  # Ensure this function computes accuracy, precision, recall, and F1
)

# Train the baseline model
trainer_baseline.train()

# Save the baseline model
trainer_baseline.save_model("./baseline_model")


model.safetensors: 100%|██████████| 542M/542M [01:04<00:00, 8.35MB/s] 
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 33%|███▎      | 4/12 [01:45<03:08, 23.56s/it]

{'loss': 2.7127, 'grad_norm': 1.5092084407806396, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


                                              
 33%|███▎      | 4/12 [01:53<03:08, 23.56s/it] 

{'eval_loss': 2.534414291381836, 'eval_precision': 0.040135669869983036, 'eval_recall': 0.06133902329359544, 'eval_f1': 0.023910133503189653, 'eval_accuracy': 0.19053051911009697, 'eval_runtime': 7.6576, 'eval_samples_per_second': 10.186, 'eval_steps_per_second': 1.306, 'epoch': 1.0}


 67%|██████▋   | 8/12 [03:48<01:42, 25.67s/it]

...

{'loss': 2.4034, 'grad_norm': 1.0129964351654053, 'learning_rate': 0.0, 'epoch': 3.0}


                                               
100%|██████████| 12/12 [05:46<00:00, 24.14s/it]

{'eval_loss': 2.438114643096924, 'eval_precision': 0.0748533313937097, 'eval_recall': 0.073749851011053, 'eval_f1': 0.044792428413757863, 'eval_accuracy': 0.20365088419851682, 'eval_runtime': 6.6827, 'eval_samples_per_second': 11.672, 'eval_steps_per_second': 1.496, 'epoch': 3.0}


100%|██████████| 12/12 [05:56<00:00, 29.71s/it]


{'train_runtime': 356.5815, 'train_samples_per_second': 3.601, 'train_steps_per_second': 0.034, 'train_loss': 2.531620184580485, 'epoch': 3.0}


## Model Adjustment and Partial Freezing

In [16]:
def train_with_different_freezing_configs(freeze_configs, training_args, compute_metrics, train_dataset, eval_dataset):
    results = {}
    
    for config in freeze_configs:
        model_adjusted = initialize_and_freeze_model(layers_to_freeze=config)
        
        trainer = Trainer(
            model=model_adjusted,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics
        )

        config_name = "_".join(str(x) for x in config)  # Convert layer indices to a string
        print(f"Training with layers {config} frozen...")
        trainer.train()

        # Evaluate the model on the validation set
        eval_results = trainer.evaluate()
        results[f"Layers frozen: {config}"] = eval_results

        # Save the model for each configuration
        model_save_path = f"./models/model_layers_{config_name}_frozen"
        model_adjusted.save_pretrained(model_save_path)
        
        # Optionally, save the Trainer state as well
        trainer.save_state()

    return results


In [17]:
# Corrected function to initialize and freeze specified layers of the model
def initialize_and_freeze_model(layers_to_freeze):
    # Initialize the model
    model = DistilBertForTokenClassification.from_pretrained(
        'distilbert-base-multilingual-cased',
        num_labels=num_labels  # Ensure num_labels is defined correctly
    )

    # Freeze specified layers
    for name, param in model.named_parameters():
        # Freeze layers based on the provided list
        if any(f"transformer.layer.{i}." in name for i in layers_to_freeze):
            param.requires_grad = False

    return model

# Define different freezing configurations considering DistilBERT has 6 layers
freeze_configs = [
    [0],                # Freeze only the first layer
    [0, 1],             # Freeze the first two layers
    [0, 1, 2],          # Freeze the first three layers
    [0, 1, 2, 3],       # Freeze the first four layers
    [0, 1, 2, 3, 4],    # Freeze the first five layers
    [0, 1, 2, 3, 4, 5]  # Freeze all layers (for completeness of experiment)
]


# Proceed with the experiments
experiment_results = train_with_different_freezing_configs(
    freeze_configs=freeze_configs,
    training_args=training_args_baseline,
    compute_metrics=compute_metrics,
    train_dataset=reduced_tokenized_datasets['train'],
    eval_dataset=reduced_tokenized_datasets['validation']
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training with layers [0] frozen...



 17%|█▋        | 2/12 [03:51<04:48, 28.82s/it]

{'loss': 2.7242, 'grad_norm': 1.6362910270690918, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}














                                         
                                              
 17%|█▋        | 2/12 [03:59<04:48, 28.82s/it]
Checkpoint destination directory ./baseline_results\checkpoint-4 already exists and is non-empty. Saving will proceed but saved results may be invalid.


...

Training with layers [0, 1, 2] frozen...



 17%|█▋        | 2/12 [15:16<04:48, 28.82s/it]

{'loss': 2.703, 'grad_norm': 1.5147755146026611, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}













                                           

                                               
 17%|█▋        | 2/12 [15:24<04:48, 28.82s/it]
Checkpoint destination directory ./baseline_results\checkpoint-4 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 2.5697734355926514, 'eval_precision': 0.07895034661380605, 'eval_recall': 0.06430688495333214, 'eval_f1': 0.03632615090725487, 'eval_accuracy': 0.18254420992584142, 'eval_runtime': 7.7185, 'eval_samples_per_second': 10.106, 'eval_steps_per_second': 1.296, 'epoch': 1.0}


## Analysis and Comaprison

In [ ]:
# # Evaluate both models on the test set to compare performance
# baseline_results = trainer_baseline.evaluate(tokenized_datasets['test'])
# experiment_results = trainer_adjusted.evaluate(tokenized_datasets['test'])

# print("Baseline Model Performance:", baseline_results)
# print("Adjusted Model Performance:", experiment_results)
